<h1>Dados do ODB2</h1>

<b>O processo da derivação se dará da seguinte forma:</b>
<img src='explica_processo_leitura_alarmes.png'>

<ol>
    <li><b>df_exp</b> - É o nosso dataset original;</li>
    <li><b>decode_trouble_code()</b> - Vai ser nossa função para a derivação, fatiar o código OBDII em linhas diferentes;</li>
    <li><b>pega_erro()</b> - Baseado no nosso dicionário em csv, consultaremos erro por erro para entender o erro em questão</li>
<ol>

In [12]:
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import datetime
import plotly.express as xp
import plotly.graph_objects as go
import matplotlib
import os

# configurando pandas
pd.set_option('display.max_columns',None)

lista_colunas = ['AIR_INTAKE_TEMP', 'AMBIENT_AIR_TEMP', 'AUTOMATIC','BAROMETRIC_PRESSURE', 'CAR_YEAR', 
                 'DAYS_OF_WEEK', 'DTC_NUMBER','ENGINE_COOLANT_TEMP', 'ENGINE_LOAD', 'ENGINE_POWER', 'ENGINE_RPM',
                 'ENGINE_RUNTIME', 'EQUIV_RATIO','FUEL_LEVEL','FUEL_TYPE', 'HOURS', 'INTAKE_MANIFOLD_PRESSURE',
                 'LONG TERM FUEL TRIM BANK 2', 'MAF', 'MARK','MIN', 'MODEL', 'MONTHS',
                 'SHORT TERM FUEL TRIM BANK 1','SHORT TERM FUEL TRIM BANK 2', 'SPEED','THROTTLE_POS', 'TIME', 
                 'TIMING_ADVANCE', 'TROUBLE_CODES','VEHICLE_ID', 'YEAR']
# Funções usadas pelo script
def format_timestamp(timestamp):
    timestamp = timestamp / 1000
    return datetime.fromtimestamp(timestamp)

def log(mes):
    print(datetime.now().strftime("[%d/%m/%Y %H:%M:%S] - "), mes)

def porcentagem(parte,total):
    perc = parte/total*100
    return str(perc)+'%'
    
def dict_odb2():
    # Leitura das bases
    df_codes = pd.read_csv("../data/odb2_codes.csv", sep=';')
    df_codes.drop(index = df_codes[(df_codes["problems"]=="ISO/SAE Reserved___") | 
                                  ((df_codes["problems"]=="___"))].index,inplace=True, axis=0)
    df_codes = df_codes[["code","problems"]]

    return df_codes.problems.str.replace('_','; ').str.replace(' ;','')

def get_experimental_dataset():
        #Leitura dos dados experimentais
    df_exp1 = pd.read_excel("../data/exp1_14drivers_14cars_dailyRoutes.xlsx")
    df_exp1.rename({"TIMESTAMP":"TIME","BAROMETRIC_PRESSURE(KPA)":"BAROMETRIC_PRESSURE"},inplace=True,
                   axis="columns")

    df_exp2 = pd.read_excel("../data/exp2_19drivers_1car_1route.xlsx")
    #, 'Short Term Fuel Trim Bank 1'
    df_exp2.rename({'Short Term Fuel Trim Bank 2':'SHORT TERM FUEL TRIM BANK 2',
                    'Short Term Fuel Trim Bank 1':'SHORT TERM FUEL TRIM BANK 1',
                    'Long Term Fuel Trim Bank 2':'LONG TERM FUEL TRIM BANK 2'},inplace=True,axis="columns")

    df_exp2.ENGINE_RPM = df_exp2.ENGINE_RPM.str.replace("RPM","").notna().astype(int)

    df_exp3 = pd.read_excel("../data/exp3_4drivers_1car_1route.xlsx")
    
    return pd.concat([df_exp1,df_exp2,df_exp3],sort=True).reset_index()

def get_problems(df):
    df["PROBLEM_1"] = df['TROUBLE_CODES'].str.slice(0,5)
    df["PROBLEM_2"] = df['TROUBLE_CODES'].str.slice(5,10)
    df["PROBLEM_3"] = df['TROUBLE_CODES'].str.slice(10,15)
    
    return df

def decode_trouble_code(df_exp):
    df_problems_final = pd.DataFrame()
    for i in df_exp.index:
        tmp2 = df_exp[df_exp.index==i]
        if str(tmp2.iloc[0]["TROUBLE_CODES"]) != 'nan':
            #print("Processando carro com problema")
            df_problems = tmp2[['PROBLEM_1','PROBLEM_2','PROBLEM_3']].T.reset_index(drop=True).reset_index()
            df_problems.columns = ['index','ERROR CODE']
            df_problems = df_problems.merge(pd.concat([tmp2,tmp2,tmp2]).reset_index(drop=True).reset_index(),on='index').drop(columns=['index','PROBLEM_1','PROBLEM_2','PROBLEM_3', 'TROUBLE_CODES'], axis=1)
            df_problems_final = pd.concat([df_problems_final, df_problems])
            log(porcentagem(i, df_exp.shape[0])+'% Concluído')
            os.system('cls')
    
    df_problems_final.reset_index(drop=True,inplace=True)
    return df_problems_final

def get_colunas_viaveis(df_exp):
    
    lista_colunas = []

    for c in df_exp.columns:
        perc = df_exp[c].isna().sum()/df_exp.shape[0]*100
        if perc < 60:
            lista_colunas.append(c)
            
    lista_colunas.append('ERROR CODE')
    return lista_colunas

def there_is_a_error(code):
    if 'P' in code :
        return True
    else:
        return False

def export_for_guesser(df_ext):
    df_ford_guesser = df_ext.copy()
    df_ford_guesser['ALERT'] = df_ford_guesser['ERROR CODE'].apply(there_is_a_error)
    df_ford_guesser['ERROR CODE'] = df_ford_guesser['ERROR CODE'].fillna('No Error Code')
    df_ford_guesser.reset_index(drop=True).to_csv('../data/ford_guesser_fuel.csv')
    
    
def export_for_classifier(df_ext):
    df_ford_classifier = df_ext[~df_ext['ERROR CODE'].isna()].reset_index(drop=True).copy()
    df_codes.columns = ['ERROR CODE', 'problems']
    df_codes['ERROR CODE'] = df_codes['ERROR CODE'].str[:5]
    df_ford_classifier = df_ford_classifier.merge(df_codes, on='ERROR CODE')
    df_ford_classifier.reset_index(drop=True).to_csv('../data/ford_classifier_fuel.csv')
    
def export_for_tear(df_ext):
    df_desgaste = df.copy()
    df.reset_index(drop=True).to_csv('../data/historico_experimentos.csv')
    

if __name__ == "__main__":
    df_codes = dict_odb2()
    df_exp = get_experimental_dataset()[lista_colunas]
    log('Dados Carregados')
    df = get_problems(df_exp)
    log('Decodificando códigos')
    df = decode_trouble_code(df)
    df = df[get_colunas_viaveis(df)]
    log('Convertendo TIMESTAMP para formato de data')
    df.TIME = format_timestamp(df.TIME.astype(int).values)
    export_for_guesser(df)
    export_for_tear(df)
    export_for_classifier(df)

[02/11/2020 14:27:23] -  Dados Carregados
[02/11/2020 14:27:24] -  Decodificando códigos
[02/11/2020 14:27:40] -  30.65649014221635%% Concluído
[02/11/2020 14:27:40] -  30.65822872839807%% Concluído
[02/11/2020 14:27:40] -  30.659967314579784%% Concluído
[02/11/2020 14:27:41] -  30.6617059007615%% Concluído
[02/11/2020 14:27:41] -  30.66344448694322%% Concluído
[02/11/2020 14:27:41] -  30.665183073124936%% Concluído
[02/11/2020 14:27:41] -  30.666921659306652%% Concluído
[02/11/2020 14:27:41] -  30.668660245488365%% Concluído
[02/11/2020 14:27:41] -  30.670398831670088%% Concluído
[02/11/2020 14:27:41] -  30.672137417851804%% Concluído
[02/11/2020 14:27:41] -  30.673876004033517%% Concluído
[02/11/2020 14:27:41] -  30.67561459021524%% Concluído
[02/11/2020 14:27:42] -  30.677353176396956%% Concluído
[02/11/2020 14:27:42] -  30.67909176257867%% Concluído
[02/11/2020 14:27:42] -  30.68083034876039%% Concluído
[02/11/2020 14:27:42] -  30.682568934942108%% Concluído
[02/11/2020 14:27:42] -

[02/11/2020 14:27:55] -  30.910323724747034%% Concluído
[02/11/2020 14:27:55] -  30.91206231092875%% Concluído
[02/11/2020 14:27:55] -  30.913800897110473%% Concluído
[02/11/2020 14:27:55] -  30.915539483292186%% Concluído
[02/11/2020 14:27:55] -  30.917278069473902%% Concluído
[02/11/2020 14:27:55] -  30.919016655655625%% Concluído
[02/11/2020 14:27:56] -  30.920755241837337%% Concluído
[02/11/2020 14:27:56] -  30.922493828019054%% Concluído
[02/11/2020 14:27:56] -  30.92423241420077%% Concluído
[02/11/2020 14:27:56] -  30.92597100038249%% Concluído
[02/11/2020 14:27:56] -  30.927709586564205%% Concluído
[02/11/2020 14:27:56] -  30.92944817274592%% Concluído
[02/11/2020 14:27:56] -  30.93118675892764%% Concluído
[02/11/2020 14:27:56] -  30.932925345109357%% Concluído
[02/11/2020 14:27:56] -  30.934663931291073%% Concluído
[02/11/2020 14:27:56] -  30.93640251747279%% Concluído
[02/11/2020 14:27:56] -  30.93814110365451%% Concluído
[02/11/2020 14:27:57] -  30.939879689836225%% Concluído

[02/11/2020 14:28:08] -  31.169373065822874%% Concluído
[02/11/2020 14:28:08] -  31.17111165200459%% Concluído
[02/11/2020 14:28:08] -  31.172850238186307%% Concluído
[02/11/2020 14:28:09] -  31.174588824368023%% Concluído
[02/11/2020 14:28:09] -  31.176327410549742%% Concluído
[02/11/2020 14:28:09] -  31.17806599673146%% Concluído
[02/11/2020 14:28:09] -  31.179804582913174%% Concluído
[02/11/2020 14:28:09] -  31.181543169094894%% Concluído
[02/11/2020 14:28:09] -  31.18328175527661%% Concluído
[02/11/2020 14:28:09] -  31.185020341458326%% Concluído
[02/11/2020 14:28:09] -  31.186758927640046%% Concluído
[02/11/2020 14:28:09] -  31.188497513821762%% Concluído
[02/11/2020 14:28:09] -  31.190236100003478%% Concluído
[02/11/2020 14:28:10] -  31.19197468618519%% Concluído
[02/11/2020 14:28:10] -  31.193713272366914%% Concluído
[02/11/2020 14:28:10] -  31.19545185854863%% Concluído
[02/11/2020 14:28:10] -  31.197190444730342%% Concluído
[02/11/2020 14:28:10] -  31.198929030912065%% Concluí

[02/11/2020 14:28:22] -  31.426683820716995%% Concluído
[02/11/2020 14:28:22] -  31.428422406898708%% Concluído
[02/11/2020 14:28:22] -  31.430160993080424%% Concluído
[02/11/2020 14:28:22] -  31.431899579262147%% Concluído
[02/11/2020 14:28:22] -  31.43363816544386%% Concluído
[02/11/2020 14:28:22] -  31.435376751625576%% Concluído
[02/11/2020 14:28:22] -  31.4371153378073%% Concluído
[02/11/2020 14:28:22] -  31.43885392398901%% Concluído
[02/11/2020 14:28:22] -  31.440592510170728%% Concluído
[02/11/2020 14:28:22] -  31.442331096352444%% Concluído
[02/11/2020 14:28:22] -  31.444069682534163%% Concluído
[02/11/2020 14:28:23] -  31.44580826871588%% Concluído
[02/11/2020 14:28:23] -  31.447546854897595%% Concluído
[02/11/2020 14:28:23] -  31.449285441079315%% Concluído
[02/11/2020 14:28:23] -  31.45102402726103%% Concluído
[02/11/2020 14:28:23] -  31.452762613442747%% Concluído
[02/11/2020 14:28:23] -  31.454501199624463%% Concluído
[02/11/2020 14:28:23] -  31.456239785806183%% Concluíd

[02/11/2020 14:28:37] -  31.683994575611113%% Concluído
[02/11/2020 14:28:37] -  31.68573316179283%% Concluído
[02/11/2020 14:28:37] -  31.68747174797455%% Concluído
[02/11/2020 14:28:37] -  31.689210334156265%% Concluído
[02/11/2020 14:28:38] -  31.69094892033798%% Concluído
[02/11/2020 14:28:38] -  31.692687506519697%% Concluído
[02/11/2020 14:28:38] -  31.694426092701416%% Concluído
[02/11/2020 14:28:38] -  31.696164678883132%% Concluído
[02/11/2020 14:28:38] -  31.69790326506485%% Concluído
[02/11/2020 14:28:38] -  31.699641851246568%% Concluído
[02/11/2020 14:28:38] -  31.701380437428284%% Concluído
[02/11/2020 14:28:38] -  31.70311902361%% Concluído
[02/11/2020 14:28:38] -  31.70485760979172%% Concluído
[02/11/2020 14:28:39] -  31.706596195973436%% Concluído
[02/11/2020 14:28:39] -  31.708334782155152%% Concluído
[02/11/2020 14:28:39] -  31.710073368336865%% Concluído
[02/11/2020 14:28:39] -  31.711811954518588%% Concluído
[02/11/2020 14:28:39] -  31.713550540700304%% Concluído
[

[02/11/2020 14:29:07] -  32.20209325776279%% Concluído
[02/11/2020 14:29:07] -  32.2038318439445%% Concluído
[02/11/2020 14:29:07] -  32.20557043012622%% Concluído
[02/11/2020 14:29:07] -  32.20730901630794%% Concluído
[02/11/2020 14:29:07] -  32.209047602489655%% Concluído
[02/11/2020 14:29:07] -  32.21078618867137%% Concluído
[02/11/2020 14:29:07] -  32.212524774853094%% Concluído
[02/11/2020 14:29:07] -  32.2142633610348%% Concluído
[02/11/2020 14:29:07] -  32.21600194721652%% Concluído
[02/11/2020 14:29:07] -  32.21774053339824%% Concluído
[02/11/2020 14:29:07] -  32.21947911957996%% Concluído
[02/11/2020 14:29:08] -  32.221217705761674%% Concluído
[02/11/2020 14:29:08] -  32.2229562919434%% Concluído
[02/11/2020 14:29:08] -  32.224694878125106%% Concluído
[02/11/2020 14:29:08] -  32.22643346430682%% Concluído
[02/11/2020 14:29:08] -  32.22817205048854%% Concluído
[02/11/2020 14:29:08] -  32.22991063667026%% Concluído
[02/11/2020 14:29:08] -  32.23164922285198%% Concluído
[02/11/20

[02/11/2020 14:29:21] -  32.46114259883863%% Concluído
[02/11/2020 14:29:21] -  32.46288118502034%% Concluído
[02/11/2020 14:29:21] -  32.46461977120206%% Concluído
[02/11/2020 14:29:21] -  32.466358357383776%% Concluído
[02/11/2020 14:29:22] -  32.46809694356549%% Concluído
[02/11/2020 14:29:22] -  32.46983552974721%% Concluído
[02/11/2020 14:29:22] -  32.471574115928924%% Concluído
[02/11/2020 14:29:22] -  32.47331270211065%% Concluído
[02/11/2020 14:29:22] -  32.47505128829236%% Concluído
[02/11/2020 14:29:22] -  32.47678987447408%% Concluído
[02/11/2020 14:29:22] -  32.478528460655795%% Concluído
[02/11/2020 14:29:22] -  32.48026704683751%% Concluído
[02/11/2020 14:29:22] -  32.48200563301923%% Concluído
[02/11/2020 14:29:23] -  32.48374421920094%% Concluído
[02/11/2020 14:29:23] -  32.48548280538267%% Concluído
[02/11/2020 14:29:23] -  32.48722139156438%% Concluído
[02/11/2020 14:29:23] -  32.4889599777461%% Concluído
[02/11/2020 14:29:23] -  32.49069856392781%% Concluído
[02/11/2

KeyboardInterrupt: 

In [7]:
int(df.TIME.head(1).values)

1502902504267

In [11]:
format_timestamp(int(df.TIME.head(1).values))

datetime.datetime(2017, 8, 16, 13, 55, 4, 267000)

In [ ]:
export_for_tear(df)
export_for_classifier(df)

<b>Apêndice A</b>
- Basicamente, os compensadores de combustível (fuel trims) são a porcentagem de mudança no combustível ao longo do tempo. Para que o motor funcione corretamente, a relação ar: combustível precisa ficar dentro de uma pequena janela de 14,7: 1. Ele deve permanecer nesta zona em todas as várias condições que um motor encontra todos os dias: partida a frio, marcha lenta em marcha lenta em trânsito intenso, cruzando na rodovia, etc;
- short term fuel trim (STFT) refere-se a mudanças imediatas no combustível que ocorrem várias vezes por segundo;
- The long term fuel trims (LTFT) são guiados pelo STFT, refere-se a mudanças no STFT, mas em média durante um longo período de tempo. Uma porcentagem de compensação de combustível negativa indica uma retirada de combustível, enquanto uma porcentagem positiva indica uma adição de combustível